In [ ]:
# zeroth shell
import pandas as pd

electron_temp = 9992.2722969523056
ff_heating_estimator = [  4.89135279e-24,   4.37696370e-24,   3.75869301e-24,
         4.97847160e-24,   4.52158002e-24,   4.21024499e-24,
         3.94991540e-24,   3.72915649e-24,   3.58902110e-24,
         3.40170224e-24,   3.20848519e-24,   3.03540032e-24,
         2.87314722e-24,   2.74328938e-24,   2.61063140e-24,
         2.50640248e-24,   2.38164559e-24,   2.26967531e-24,
         2.24509826e-24,   2.12378192e-24,   2.02063266e-24,
         1.92509873e-24,   1.83070678e-24,   1.77346374e-24]

data_path = "/home/afullard/tardis-chvogl-configs"
bf_heating_coeff = pd.read_csv(f"{data_path}/bound_free_heating_coeff.csv", index_col=(0, 1, 2))
stim_recomb_cooling_coeff = pd.read_csv(f"{data_path}/stim_recomb_cooling_coeff.csv", index_col=(0, 1, 2))
electron_densities = pd.read_csv(f"{data_path}/thermal_electron_densities.csv", index_col=0, header=None)
ion_number_density = pd.read_csv(f"{data_path}/thermal_ion_number_density.csv", index_col=(0, 1))
level_number_density = pd.read_csv(f"{data_path}/thermal_level_number_density.csv", index_col=(0, 1, 2))
phi_lucy = pd.read_csv(f"{data_path}/thermal_phi_lucy.csv", index_col=(0, 1, 2))
coll_exc_coeff = pd.read_csv(f"{data_path}/thermal_coll_exc_coeff.csv", index_col=(0, 1, 2, 3))
coll_deexc_coeff = pd.read_csv(f"{data_path}/thermal_coll_deexc_coeff.csv", index_col=(0, 1, 2, 3))

In [ ]:
#outputs

fb_cooling_ct = 1.2018593543520837e-06
ff_cooling_ct = 6.941664530316456e-07
coll_ion_cooling_ct = 1.6125333965984259e-07
bf_heating_ct = 1.2809489753862688e-06
ff_heating_ct = 2.3829164962085199e-07
coll_ion_heating_ct = 1.5946196993219911e-07
total_heating_rate_ct = -3.8611749800759465e-07
fractional_heating_rate_ct = -0.036553097452112299

col_exc_cooling_rate_ct = pd.read_csv(f"{data_path}/col_exc_cooling_rate.csv", index_col=0)
col_deexc_cooling_rate_ct = pd.read_csv(f"{data_path}/col_deexc_heating_rate.csv", index_col=0)

In [ ]:
from pathlib import Path

import astropy.units as u

from tardis.io.atom_data import AtomData
from tardis.plasma.electron_energy_distribution import (
         ThermalElectronEnergyDistribution,
)
from tardis.plasma.equilibrium.rates.heating_cooling_rates import (
         BoundFreeThermalRates,
         CollisionalBoundThermalRates,
         CollisionalIonizationThermalRates,
         FreeFreeThermalRates,
)

home = str(Path('~').expanduser())
cmfgen_atom_data = AtomData.from_hdf(home+'/tardis-regression-data/atom-data/nlte_atom_data/TestNLTE_He_Ti.h5')
cmfgen_atom_data.prepare_atom_data([1], "macroatom", [], [(1, 0)])


thermal_electron_distribution = ThermalElectronEnergyDistribution(0 * u.erg, electron_temp * u.K, electron_densities.values[0] * u.cm**-3)

In [ ]:
bf_rates = BoundFreeThermalRates(cmfgen_atom_data.photoionization_data)
bf_heating, bf_cooling = bf_rates.solve(
    level_number_density["0"],
    ion_number_density["0"],
    thermal_electron_distribution,
    phi_lucy["0"],
    bound_free_heating_estimator=bf_heating_coeff["0"],
    stimulated_recombination_estimator=stim_recomb_cooling_coeff["0"],
)

In [ ]:
(bf_heating - bf_heating_ct) / bf_heating_ct

In [ ]:
(bf_cooling - fb_cooling_ct) / fb_cooling_ct

In [ ]:
ff_rates = FreeFreeThermalRates()
ff_heating, ff_cooling = ff_rates.solve(
    ff_heating_estimator[0],
    thermal_electron_distribution,
    ion_number_density["0"],
)

In [ ]:
(ff_cooling - ff_cooling_ct) / ff_cooling_ct

In [ ]:
(ff_heating - ff_heating_ct) / ff_heating_ct

In [ ]:
coll_ion_rates = CollisionalIonizationThermalRates(cmfgen_atom_data.photoionization_data)
coll_ion_heating, coll_ion_cooling = coll_ion_rates.solve(
    thermal_electron_distribution.number_density,
    ion_number_density,
    level_number_density,
    collisional_ionization_rate_coefficient,
    phi_lucy,
    )

In [ ]:
# need to get data with collision rates
coll_bound_rates = CollisionalBoundThermalRates(cmfgen_atom_data.collision_data)
coll_deexc_heating, coll_exc_cooling = coll_bound_rates.solve(
    thermal_electron_distribution.number_density,
    coll_deexc_coeff["0"],
    coll_exc_coeff["0"],
    level_number_density["0"],
)